In [ ]:
#@title License information { display-mode: "form" }
#@markdown Copyright 2024 The MediaPipe Authors.
#@markdown Licensed under the Apache License, Version 2.0 (the "License");
#@markdown
#@markdown you may not use this file except in compliance with the License.
#@markdown You may obtain a copy of the License at
#@markdown
#@markdown https://www.apache.org/licenses/LICENSE-2.0
#@markdown
#@markdown Unless required by applicable law or agreed to in writing, software
#@markdown distributed under the License is distributed on an "AS IS" BASIS,
#@markdown WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#@markdown See the License for the specific language governing permissions and
#@markdown limitations under the License.

In [ ]:
#@title Setup { display-mode: "form" }
!pip install --upgrade numpy


import ipywidgets as widgets
from IPython.display import display
from google.colab import files
install_out = widgets.Output()
display(install_out)
with install_out:
  !pip install --upgrade numpy
  !pip install --upgrade scipy
  !pip install mediapipe
  from mediapipe.tasks.python.genai import bundler

install_out.clear_output()
with install_out:
  print("Setup done.")

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-cased-distilled-squad")
tokenizer.save_pretrained("bert_tokenizer/")


In [ ]:
# build_fake_corpus.py
with open("bert_tokenizer/vocab.txt", "r") as fin, open("fake_corpus.txt", "w") as fout:
    for line in fin:
        token = line.strip()
        if token and not token.startswith("["):
            fout.write(token.replace("##", "") + "\n")


In [ ]:
!pip install sentencepiece


In [ ]:
!pip install tf-keras # Install the compatible Keras for transformers
!pip install --upgrade tensorflow # Ensure TensorFlow is upgraded
!pip install tensorflow --no-deps # Reinstall TensorFlow without dependencies
from transformers import TFAutoModelForQuestionAnswering
import tensorflow as tf # Import tensorflow

# Instead of importing from tf_keras, import from tensorflow.keras.activations
from tensorflow.keras import activations

model = TFAutoModelForQuestionAnswering.from_pretrained("distilbert-base-cased-distilled-squad")
model.save("distilbert_tf_model")

converter = tf.lite.TFLiteConverter.from_saved_model("distilbert_tf_model") # Use tf.lite
tflite_model = converter.convert()

# Save the model to a file
with open("distilbert-qa.tflite", "wb") as f:
    f.write(tflite_model)

In [ ]:
!find / -name "spm_train" 2>/dev/null

In [ ]:
!apt-get update && apt-get install -y cmake build-essential pkg-config libgoogle-perftools-dev
!git clone https://github.com/google/sentencepiece.git
%cd sentencepiece
!mkdir build && cd build && cmake .. && make -j $(nproc) && make install && ldconfig
%cd /content

In [ ]:
!spm_train --version

In [ ]:
!spm_train \
      --input=fake_corpus.txt \
      --model_prefix=distilbert_tokenizer \
      --vocab_size=26598 \
      --character_coverage=1.0 \
      --model_type=word

In [ ]:
#!pip install --upgrade --force-reinstall mediapipe-model-maker==0.2.1.4
#!pip install --upgrade  --user mediapipe-model-maker==0.2.1.4
!python3 -m mediapipe_model_maker.genai.task_converter \
  --tflite_model distilbert-qa.tflite \
  --tokenizer_model distilbert_tokenizer.model \
  --output_file distilbert_qa.task

In [ ]:
!pip install mediapipe
!pip install --upgrade mediapipe-model-maker # Install or upgrade mediapipe-model-maker
!pip install sentencepiece  # Install or upgrade sentencepiece


In [ ]:
#!pip install tensorflow text
#!pip install tensorflow_hub
#!pip uninstall mediapipe-model-maker
#!pip install --upgrade mediapipe-model-maker

#!pip uninstall mediapipe-model-maker
#!pip install --upgrade --force-reinstall mediapipe-model-maker==0.2.1.4

!find / -name "mediapipe_model_maker" 2>/dev/null
!find / -name "model_maker" 2>/dev/null


import site
import os

for path in site.getsitepackages():
    package_path = os.path.join(path, 'mediapipe_model_maker')
    if os.path.exists(package_path):
        print(f"Found mediapipe_model_maker at: {package_path}")
        break  # Stop searching once found
else:
    print("mediapipe_model_maker not found in site-packages.")


!ls {package_path}
!ls {package_path}/genai  # Assuming genai subfolder exists

In [ ]:
#import tensorflow as tf; print(tf.__version__)

!ls /usr/local/lib/python3.11/dist-packages/mediapipe_model_maker
!ls /usr/local/lib/python3.11/dist-packages/mediapipe_model_maker/genai

# Why do we need task bundles?

Executing a text generation pipeline in its entirety requires more than merely employing the core transformer model. It requires preparing the input text to align with the model's required format, running the model autoregressively, and sampling during each iteration. Consequently, once the user has converted their model into `tflite` format (comprising the model's graph and parameters), it becomes essential to augment it with additional metadata to ensure successful end-to-end execution of the model.

Task bundler offers a practical approach to creating such bundles. The example below illustrates how a task bundle can be generated for a converted Gemma model.

In [3]:
from mediapipe.tasks.python.genai import bundler

tflite_model="distilbert-qa.tflite" # @param {type:"string"}
tokenizer_model="distilbert_tokenizer.model" # @param {type:"string"}
start_token="<bos>" # @param {type:"string"}
stop_token="<eos>" # @param {type:"string"}
output_filename="distilbert.task" # @param {type:"string"}
enable_bytes_to_unicode_mapping=False # @param ["False", "True"] {type:"raw"}

config = bundler.BundleConfig(
    tflite_model=tflite_model,
    tokenizer_model=tokenizer_model,
    start_token=start_token,
    stop_tokens=[stop_token],
    output_filename=output_filename,
    enable_bytes_to_unicode_mapping=enable_bytes_to_unicode_mapping,
)
bundler.create_bundle(config)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title Download the Bundle { display-mode: "form" }
#@markdown Run this cell to download the generated `.task` file.
files.download(output_filename)

**Notes:**

* The current task pipeline only supports SentencePiece tokenizer models.
* Certain models (e.g., phi-2) use bytes to unicode mapping. Use `enable_bytes_to_unicode_mapping` flag accordingly. Such information, including `start_token` and `stop_tokens` are often provided along with model artifacts.
* The generated output bundle file must end with `.task`. If not, `create_bundle` automatically adds the extension. Do not remove or change that.

In [12]:
import tensorflow as tf
import numpy as np
model_path = "distilbert-qa.tflite"  # Replace with your model

interpreter = tf.lite.Interpreter(model_path=model_path)
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print("Input Tensor Details:", input_details)
print("Output Tensor Details:", output_details)

# Assuming your model has two inputs: input_ids and attention_mask
# Adjust the input shapes and data types accordingly
input_shape_ids = input_details[0]['shape']
input_data_ids = np.random.randint(0, 30522, size=input_shape_ids).astype(np.int32) # Generate random token IDs

input_shape_mask = input_details[1]['shape']
input_data_mask = np.ones(input_shape_mask).astype(np.int32) # Create an attention mask of all 1s

# Set the tensors for both inputs
interpreter.set_tensor(input_details[0]['index'], input_data_ids)
interpreter.set_tensor(input_details[1]['index'], input_data_mask)

interpreter.invoke()

output_index = output_details[0]['index']
output_data = interpreter.get_tensor(output_index)

print("Raw Output Data:", output_data)

# Add your post-processing here (if needed)
# ...

Input Tensor Details: [{'name': 'serving_default_attention_mask:0', 'index': 0, 'shape': array([1, 1], dtype=int32), 'shape_signature': array([-1, -1], dtype=int32), 'dtype': <class 'numpy.int32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}, {'name': 'serving_default_input_ids:0', 'index': 1, 'shape': array([1, 1], dtype=int32), 'shape_signature': array([-1, -1], dtype=int32), 'dtype': <class 'numpy.int32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
Output Tensor Details: [{'name': 'StatefulPartitionedCall:0', 'index': 724, 'shape': array([1, 1], dtype=int32), 'shape_signature': array([-1, -1], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': arra